In [ ]:
import json
import pandas
import pathlib
import pydash
import subprocess

dataframe = pandas.read_csv(pathlib.Path.cwd() / 'pike_cooper.csv')

if not (pathlib.Path.cwd() / 'acmi-api').exists():
    subprocess.call(['git', 'clone', 'https://github.com/ACMILabs/acmi-api.git', '--depth', '1'])

acmi_path = pathlib.Path.cwd() / 'acmi-api' / 'app' / 'json' / 'works'
acmi_files = [x for x in acmi_path.iterdir() if x.suffix == '.json']

matched_holdings = pandas.DataFrame(columns=['wikidata_id', 'acmi_holdings'])

for x in acmi_files:
    with open(x) as data:
        data = json.load(data)
    if 'external_references' in data and 'id' in data:
        for y in [y for y in data['external_references'] if pydash.get(y, 'source.name') == 'Wikidata']:
            if y['source_identifier'] in dataframe.wikidata_id.unique():
                acmi_holdings = [z['name'] for z in data['holdings']]
                if acmi_holdings:
                    matched_holdings.loc[len(matched_holdings)] = [(y['source_identifier']), (acmi_holdings)]
                    
dataframe = pandas.merge(dataframe, matched_holdings, on='wikidata_id', how='left')
dataframe['acmi_holdings'] = dataframe['acmi_holdings'].fillna('')

# print(json.dumps(dataframe.to_dict('records'), indent=4))